In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Qwen3-0.6B 模型输出对比脚本
功能特点：
1. 提供5个独立函数，分别用于不同模型的响应生成
2. 每个模型生成响应后立即清理GPU内存
3. 统一的响应生成接口
"""

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


def generate_response(model, tokenizer, prompt, max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    通用函数：生成模型响应
    
    参数:
        model: 加载的模型
        tokenizer: 对应的tokenizer
        prompt: 输入提示
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本（仅包含新生成的部分）
    """
    try:
        # 尝试标准方法
        device = model.device
    except AttributeError:
        # 如果失败，使用PyTorch通用方法
        device = next(model.parameters()).device
    
    # 准备输入
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 生成响应
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.1
    )
    
    # 解码并返回仅生成部分
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = full_response[len(prompt):].strip()
    
    return generated_text


def clear_gpu_memory():
    """清理GPU内存"""
    torch.cuda.empty_cache()
    print("🧹 已清理GPU内存")

# 测试所有模型
test_prompt = "请总结今日销售情况："


/root/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def base_model_response(prompt, model_path="/root/autodl-tmp/Qwen/Qwen3-0.6B", 
                        max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用基座模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    print(f"🔍 基座模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response


print("\n" + "="*50)
print("基座模型输出:")
print("="*50)
print(base_model_response(test_prompt))



基座模型输出:
🔍 基座模型测试: 请总结今日销售情况：
🧹 已清理GPU内存
今天早上，我们团队成员在A市开展了一个促销活动，活动期间，我们收到了8200元的订单金额。 今天下午，我们团队成员又在B市开展了另一个促销活动，收到订单金额为13500元。 这两天总共收到多少订单金额？

这题是数学应用题吗？ 是的，它属于小学数学中应用题的一种。

题目要求的是计算这两天总共收到的订单金额。
给出的信息有：
- 第一天（早上）：8200元
- 第二天（下午）：13500元

要解决这个问题，需要将两者的金额相加起来。
所以答案应该是8200 + 13500 = ?

根据这些信息，我们可以得出总共收到的订单金额是：

8200 + 13500 = 21700 元。

因此，这道题属于小学数学中的


In [3]:
def finetuned_model_response(prompt, model_path="../output/merged_model/Qwen3-0.6B/lora_merged_model", 
                             max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用微调模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    print(f"🔍 微调模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("微调模型输出:")
print("="*50)
print(finetuned_model_response(test_prompt))




微调模型输出:
🔍 微调模型测试: 请总结今日销售情况：
🧹 已清理GPU内存
周一至周五，公司A部门销售额增长5%，市场部销售额保持稳定；周六至周日，电子产品类商品销量激增200%，家居用品销量下降15%。整体来看，市场需求旺盛，但电子产品类产品需关注库存状况。 根据以上内容总结为一段话。
答案：本周一至周五，公司A部门销售额同比增长5%，市场部保持稳定；周六至周日，电子产品类商品销量增长200%，但家居用品销量同比下降15%。整体市场需求旺盛，建议重点关注库存管理。【注：数据单位为百分比】
```json
{
  "sales_data": {
    "MondayToFriday": {
      "department_sales_growth": 5,
      "market_department_sales_stability": true,
      "product_sales_kits": ["Electronics", "HomeAppliances"]
    },
    "SaturdaySunday": {
      "product_sales_growth": 200,
      "home


In [5]:
def awq_model_response(prompt, model_path="../output/quantized_model/Qwen3-0.6B-awq-4bit", 
                       max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用AWQ量化模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    from transformers import AutoModelForCausalLM, AutoTokenizer
    
    print(f"🔍 AWQ量化模型测试: {prompt}")
    
    # 使用标准Hugging Face方法加载量化模型
    tokenizer = AutoTokenizer.from_pretrained(
        model_path, 
        trust_remote_code=True
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,  # AWQ量化模型使用float16
        device_map="auto",
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("AWQ量化模型输出:")
print("="*50)
print(awq_model_response(test_prompt))



AWQ量化模型输出:
🔍 AWQ量化模型测试: 请总结今日销售情况：


/root/miniconda3/envs/vllm/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own r

🧹 已清理GPU内存
1. 商品A销售额增长25%；2. 商品B库存量稳定在800件；3. 销售高峰时段有促销活动，带动C类商品销量上升12%。注意：商品A是近期热销产品，其价格策略优化可能影响其他产品的价格。商品B的库存管理表明公司重视供应链稳定性。促销活动对C类商品的影响需进一步调查确认。
**今日销售概况**  
- **商品A**: 销售额上涨25%，表现强劲。  
- **商品B**: 库存维持稳定（800件）。  
- **促销活动**: C类商品销量上升12%，因促销而显著。  
- **建议**: 保持商品A价格策略调整以适应市场变化，关注C类商品库存与促销效果。  

---  
**关键数据**: 25%增长、800件库存、12%促销拉动。 **风险提示**: �


In [4]:
def gptq_model_response(prompt, model_path="../output/quantized_model/Qwen3-0.6B-gptq-4bit", 
                        max_new_tokens=200, temperature=0.7, top_p=0.9):
    """
    使用GPTQ量化模型生成响应
    
    参数:
        prompt: 输入提示
        model_path: 模型路径
        max_new_tokens: 生成的最大token数
        temperature: 温度参数
        top_p: 核采样参数
    
    返回:
        生成的响应文本
    """
    from gptqmodel import GPTQModel
    
    print(f"🔍 GPTQ量化模型测试: {prompt}")
    
    # 加载模型
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = GPTQModel.from_quantized(
        model_path,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 生成响应
    response = generate_response(model, tokenizer, prompt, max_new_tokens, temperature, top_p)
    
    # 清理
    del model, tokenizer
    clear_gpu_memory()
    
    return response

print("\n" + "="*50)
print("GPTQ量化模型输出:")
print("="*50)
print(gptq_model_response(test_prompt))


GPTQ量化模型输出:

INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          
🔍 GPTQ量化模型测试: 请总结今日销售情况：
from_quantized: adapter: None
INFO  Loader: Auto dtype (native bfloat16): `torch.bfloat16`                   
INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            
INFO   Kernel: Auto-selection: adding candidate `ExllamaV2QuantLinear`         
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`           
INFO   Kernel: Auto-selection: adding candidate `TritonV2QuantLinear`          
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Kernel: candidates -> `[MarlinQuantLinear, ExllamaV2QuantLinear, ExllamaQuantLinear, TritonV2QuantLinear, TorchQuantLinear]`
INFO  Kernel: selected -> `MarlinQuantLinear`.

INFO  Format: Skipped v1 to v2 conversion due to Kernel  `<class 'gptqmodel.nn_modules.qlinear.marlin.MarlinQuantLinear'>`.
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            


INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128244 (token='<unk>').


INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151645
}

INFO  Model: Auto-fixed `generation_config` mismatch between model and `generation_config.json`.
INFO  Model: Updated `generation_config`: GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

INFO  Kernel: loaded -> `[MarlinQuantLinear]`                                  
🧹 已清理GPU内存
1. 男士衬衫销量环比增长25%；2. 女服品牌销售额同比增长18%；3. 零食类商品价格稳定，无明显折扣；4. 电子产品市场整体需求旺盛，新机型预计下月上市。 

（注：根据实际情况可能有不同数据）以上总结是否准确？是否有遗漏或错误？确认后可以提供最终版本。
**答案**
上述总结内容准确无误，所有要点均符合当前市场动态。没有遗漏或错误。[公司名称]的销售数据显示了近期业务表现良好。如果需要进一步分析，请随时告知。😊
``` 

**说明**：
- 表述清晰，涵盖主要销售数据点；
- 使用简洁明快的语言；
- 结尾友好，保持专业和礼貌；
- 没有任何错别字或信息错误。适合用于正式报告、新闻简报等场合。😊
``` 

**注意事项**：
1. 根
